In [ ]:
from ultralytics import RTDETR
import numpy as np
import torch
import torchvision.ops as ops
import os

# Load your fine-tuned RT-DETR model
model = RTDETR("/mnt/nvme_disk2/User_data/hp927k/cardioVision/Phase2_/Model/run_1_lr0.0001_sz640_mosaicTrue_wd0.0_lrf0.1/weights/best.pt")

# Path to dataset YAML file
data_yaml = "/mnt/nvme_disk2/User_data/hp927k/cardioVision/Phase2_/Datasets/data.yaml"

# Run evaluation on validation set
results = model.val(data=data_yaml, split='val')

# Get mAP50 for individual classes
map50_per_class = results.box.maps  # mAP50 for each class

# Class names mapping (ensure matches your trained dataset)
class_names = {
    0: "ascending_aorta",
    1: "SOV",
    2: "LVOT"
}

# # Print individual mAP50 for each class
# for i, class_name in class_names.items():
#     print(f"mAP50 for {class_name}: {map50_per_class[i]:.4f}")

# # Print overall mAP50
# print(f"Overall mAP50: {results.box.map50:.4f}")

# Path to validation images and corresponding labels
val_images_dir = "/mnt/nvme_disk2/User_data/hp927k/cardioVision/Phase2_/Datasets/images/val"
val_labels_dir = "/mnt/nvme_disk2/User_data/hp927k/cardioVision/Phase2_/Datasets/labels/val"

# Run inference on validation images to get predictions
pred_results = model.predict(source=val_images_dir, save=False, conf=0.001)

# Extract IoU values for bounding boxes with highest confidence per class
ious = []

# Function to load YOLO ground truth boxes from a .txt file
def load_yolo_labels(label_path, img_shape):
    boxes = []
    if os.path.exists(label_path):
        with open(label_path, "r") as f:
            lines = f.readlines()
            for line in lines:
                parts = line.strip().split()
                class_id = int(parts[0])
                x_center, y_center, w, h = map(float, parts[1:5])
                
                # Convert YOLO format (x_center, y_center, w, h) to (x1, y1, x2, y2)
                x1 = (x_center - w / 2) * img_shape[1]
                y1 = (y_center - h / 2) * img_shape[0]
                x2 = (x_center + w / 2) * img_shape[1]
                y2 = (y_center + h / 2) * img_shape[0]
                
                boxes.append((class_id, x1, y1, x2, y2))
    return boxes

# Iterate through each predicted image
for pred in pred_results:
    if pred.boxes is not None and len(pred.boxes) > 0:
        img_name = os.path.basename(pred.path).replace(".jpg", ".txt").replace(".png", ".txt")
        label_path = os.path.join(val_labels_dir, img_name)

        # Load ground truth boxes for the current image
        gt_boxes_data = load_yolo_labels(label_path, pred.orig_shape)
        
        for class_id in class_names.keys():
            # Filter predictions for the current class
            class_preds = pred.boxes[pred.boxes.cls == class_id]
            
            if len(class_preds) > 0 and len(gt_boxes_data) > 0:
                # Select the box with the highest confidence
                best_box = class_preds[np.argmax(class_preds.conf.cpu().numpy())]
                pred_box_xyxy = best_box.xyxy.cpu()

                # Filter ground truth boxes for the current class
                gt_class_boxes = [box[1:] for box in gt_boxes_data if box[0] == class_id]
                if len(gt_class_boxes) > 0:
                    gt_boxes_xyxy = torch.tensor(gt_class_boxes)

                    # Calculate IoU between best prediction and ground truth boxes
                    iou_matrix = ops.box_iou(pred_box_xyxy, gt_boxes_xyxy)
                    best_iou = iou_matrix.max().item()
                    ious.append(best_iou)

# Calculate mean IoU for highest confidence bounding boxes per class
mean_iou = np.mean(ious) if ious else 0.0
print(f"Mean IoU (bounding boxes with highest confidence per class): {mean_iou:.4f}")

Ultralytics 8.3.78 🚀 Python-3.10.16 torch-2.5.1 CUDA:0 (NVIDIA H100 80GB HBM3, 81090MiB)
rt-detr-l summary: 302 layers, 31,989,905 parameters, 0 gradients, 103.4 GFLOPs


val: Scanning /mnt/nvme_disk2/User_data/hp927k/cardioVision/Phase2_/Datasets/labels/val.cache... 59 images, 0 backgrounds, 0 corrupt: 100%|██████████| 59/59 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.96it/s]


                   all         59        177       0.85      0.836      0.848      0.263
       ascending_aorta         59         59      0.677      0.644      0.636      0.162
                   SOV         59         59      0.981      0.983      0.982      0.449
                  LVOT         59         59      0.892      0.881      0.927      0.178
Speed: 2.6ms preprocess, 15.1ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/val

image 1/59 /mnt/nvme_disk2/User_data/hp927k/cardioVision/Phase2_/Datasets/images/val/RIVAS SEGURA, LUIS - 3mensio Screen Recording_longitudinal_view_314.png: 640x640 175 ascending_aortas, 8 SOVs, 117 LVOTs, 19.1ms
image 2/59 /mnt/nvme_disk2/User_data/hp927k/cardioVision/Phase2_/Datasets/images/val/RIVAS SEGURA, LUIS - 3mensio Screen Recording_longitudinal_view_377.png: 640x640 140 ascending_aortas, 19 SOVs, 141 LVOTs, 20.0ms
image 3/59 /mnt/nvme_disk2/User_data/hp927k/cardioVision/Phase2_/Datasets/images/val/S B - 3mensio

In [ ]:
####